In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1038743,2021-02-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1038744,2021-02-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1038745,2021-03-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1038746,2021-03-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
28155,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28157,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28159,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28161,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28163,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
643203,2021-02-27,Arkansas,Arkansas,5001,1980,38.00,5000,Arkansas,AR,Arkansas,State,3017804
643205,2021-02-28,Arkansas,Arkansas,5001,1993,33.00,5000,Arkansas,AR,Arkansas,State,3017804
643207,2021-03-01,Arkansas,Arkansas,5001,1993,33.00,5000,Arkansas,AR,Arkansas,State,3017804
643209,2021-03-02,Arkansas,Arkansas,5001,1996,33.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1038743,2021-02-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1038744,2021-02-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1038745,2021-03-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1038746,2021-03-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-03') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
407,2021-03-03,Snohomish,Washington,31131,541.00,53061,WA,County,822083,3786.84,65.81
812,2021-03-03,Cook,Illinois,476094,9961.00,17031,IL,County,5150233,9244.13,193.41
1216,2021-03-03,Orange,California,261798,3966.00,6059,CA,County,3175692,8243.81,124.89
1619,2021-03-03,Maricopa,Arizona,512547,9172.00,4013,AZ,County,4485414,11426.97,204.49
2022,2021-03-03,Los Angeles,California,1196552,21669.00,6037,CA,County,10039107,11918.91,215.85
...,...,...,...,...,...,...,...,...,...,...,...
1038303,2021-03-03,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1038445,2021-03-03,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1038556,2021-03-03,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1038663,2021-03-03,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
407,2021-03-03,Snohomish,Washington,31131,541.00,53061,WA,County,822083,3786.84,65.81,65.81,3786.84
812,2021-03-03,Cook,Illinois,476094,9961.00,17031,IL,County,5150233,9244.13,193.41,193.41,9244.13
1216,2021-03-03,Orange,California,261798,3966.00,6059,CA,County,3175692,8243.81,124.89,124.89,8243.81
1619,2021-03-03,Maricopa,Arizona,512547,9172.00,4013,AZ,County,4485414,11426.97,204.49,204.49,11426.97
2022,2021-03-03,Los Angeles,California,1196552,21669.00,6037,CA,County,10039107,11918.91,215.85,215.85,11918.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038303,2021-03-03,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1038445,2021-03-03,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1038556,2021-03-03,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1038663,2021-03-03,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
731693,2021-03-03,Clarke,Alabama,3439,53.00,1025,AL,County,23622,14558.46,224.37,224.37,14558.46,1
672448,2021-03-03,Hale,Alabama,2119,69.00,1065,AL,County,14651,14463.18,470.96,470.96,14463.18,2
526581,2021-03-03,Lowndes,Alabama,1325,51.00,1085,AL,County,9726,13623.28,524.37,524.37,13623.28,3
467075,2021-03-03,Etowah,Alabama,13379,320.00,1055,AL,County,102268,13082.29,312.90,312.90,13082.29,4
392136,2021-03-03,Franklin,Alabama,3997,78.00,1059,AL,County,31362,12744.72,248.71,248.71,12744.72,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641850,2021-03-03,Converse,Wyoming,969,16.00,56009,WY,County,13822,7010.56,115.76,115.76,7010.56,19
642191,2021-03-03,Sublette,Wyoming,678,7.00,56035,WY,County,9831,6896.55,71.20,71.20,6896.55,20
886671,2021-03-03,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,21
797215,2021-03-03,Lincoln,Wyoming,1242,11.00,56023,WY,County,19830,6263.24,55.47,55.47,6263.24,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
526581,2021-03-03,Lowndes,Alabama,1325,51.00,1085,AL,County,9726,13623.28,524.37,524.37,13623.28,3,1
672448,2021-03-03,Hale,Alabama,2119,69.00,1065,AL,County,14651,14463.18,470.96,470.96,14463.18,2,2
465699,2021-03-03,Clay,Alabama,1455,54.00,1027,AL,County,13235,10993.58,408.01,408.01,10993.58,20,3
224468,2021-03-03,Walker,Alabama,6571,259.00,1127,AL,County,63521,10344.61,407.74,407.74,10344.61,31,4
567738,2021-03-03,Greene,Alabama,885,32.00,1063,AL,County,8111,10911.11,394.53,394.53,10911.11,23,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566369,2021-03-03,Hot Springs,Wyoming,361,3.00,56017,WY,County,4413,8180.38,67.98,67.98,8180.38,17,19
777450,2021-03-03,Uinta,Wyoming,2066,12.00,56041,WY,County,20226,10214.58,59.33,59.33,10214.58,4,20
797215,2021-03-03,Lincoln,Wyoming,1242,11.00,56023,WY,County,19830,6263.24,55.47,55.47,6263.24,22,21
223418,2021-03-03,Teton,Wyoming,3364,9.00,56039,WY,County,23464,14336.86,38.36,38.36,14336.86,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.81,3786.84,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.81,3786.84,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.81,3786.84,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.81,3786.84,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.81,3786.84,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036293,2021-02-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1036294,2021-02-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1036295,2021-03-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1036296,2021-03-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
728907,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14558.46,39,1,2.00
728908,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14558.46,39,1,0.00
728909,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,224.37,14558.46,39,1,1.00
728910,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,224.37,14558.46,39,1,4.00
728911,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,224.37,14558.46,39,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810459,2021-02-27,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5511.60,7,23,0.00
810460,2021-02-28,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5511.60,7,23,0.00
810461,2021-03-01,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5511.60,7,23,0.00
810462,2021-03-02,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,1.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
524471,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13623.28,1,3,1.00,0.00
524472,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13623.28,1,3,0.00,0.00
524473,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13623.28,1,3,0.00,0.00
524474,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13623.28,1,3,0.00,0.00
524475,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13623.28,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522751,2021-02-27,Albany,Wyoming,3901,11.00,56001,WY,County,38880,10033.44,28.29,28.29,10059.16,23,6,0.00,0.00
522752,2021-02-28,Albany,Wyoming,3905,11.00,56001,WY,County,38880,10043.72,28.29,28.29,10059.16,23,6,4.00,0.00
522753,2021-03-01,Albany,Wyoming,3909,11.00,56001,WY,County,38880,10054.01,28.29,28.29,10059.16,23,6,4.00,0.00
522754,2021-03-02,Albany,Wyoming,3911,11.00,56001,WY,County,38880,10059.16,28.29,28.29,10059.16,23,6,2.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-03') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
267836,2021-03-03,Imperial,California,26926,635.00,6025,CA,County,181215,14858.59,350.41,350.41,14858.59,1,2,13.00,0.00
2022,2021-03-03,Los Angeles,California,1196552,21669.00,6037,CA,County,10039107,11918.91,215.85,215.85,11918.91,2,5,1674.00,115.00
468403,2021-03-03,Inyo,California,1292,37.00,6027,CA,County,18039,7162.26,205.11,205.11,7162.26,3,27,7.00,0.00
59849,2021-03-03,Stanislaus,California,56433,953.00,6099,CA,County,550660,10248.25,173.07,173.07,10248.25,4,10,110.00,3.00
74166,2021-03-03,Tulare,California,48114,768.00,6107,CA,County,466195,10320.57,164.74,164.74,10320.57,5,9,28.00,2.00
27792,2021-03-03,Riverside,California,290498,3849.00,6065,CA,County,2470546,11758.45,155.80,155.80,11758.45,6,6,173.00,20.00
47645,2021-03-03,San Joaquin,California,67123,1150.00,6077,CA,County,762148,8807.08,150.89,150.89,8807.08,7,18,83.00,10.00
424605,2021-03-03,Merced,California,29264,407.00,6047,CA,County,277680,10538.75,146.57,146.57,10538.75,8,8,69.00,2.00
27068,2021-03-03,Fresno,California,95785,1451.00,6019,CA,County,999101,9587.12,145.23,145.23,9587.12,9,14,108.00,0.00
567680,2021-03-03,Kings,California,22091,220.00,6031,CA,County,152940,14444.23,143.85,143.85,14444.23,10,3,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
987073,2021-03-03,Lassen,California,5623,19.00,6035,CA,County,30573,18392.05,62.15,62.15,18392.05,38,1,22.00,0.00
267836,2021-03-03,Imperial,California,26926,635.00,6025,CA,County,181215,14858.59,350.41,350.41,14858.59,1,2,13.00,0.00
567680,2021-03-03,Kings,California,22091,220.00,6031,CA,County,152940,14444.23,143.85,143.85,14444.23,10,3,0.00,0.00
133594,2021-03-03,San Bernardino,California,287055,2959.00,6071,CA,County,2180085,13167.15,135.73,135.73,13167.15,12,4,241.00,7.00
2022,2021-03-03,Los Angeles,California,1196552,21669.00,6037,CA,County,10039107,11918.91,215.85,215.85,11918.91,2,5,1674.00,115.00
27792,2021-03-03,Riverside,California,290498,3849.00,6065,CA,County,2470546,11758.45,155.80,155.80,11758.45,6,6,173.00,20.00
167877,2021-03-03,Kern,California,103756,915.00,6029,CA,County,900202,11525.86,101.64,101.64,11525.86,18,7,134.00,13.00
424605,2021-03-03,Merced,California,29264,407.00,6047,CA,County,277680,10538.75,146.57,146.57,10538.75,8,8,69.00,2.00
74166,2021-03-03,Tulare,California,48114,768.00,6107,CA,County,466195,10320.57,164.74,164.74,10320.57,5,9,28.00,2.00
59849,2021-03-03,Stanislaus,California,56433,953.00,6099,CA,County,550660,10248.25,173.07,173.07,10248.25,4,10,110.00,3.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
267836,2021-03-03,Imperial,California,26926,635.00,6025,CA,County,181215,14858.59,350.41,350.41,14858.59,1,2,13.00,0.00
2022,2021-03-03,Los Angeles,California,1196552,21669.00,6037,CA,County,10039107,11918.91,215.85,215.85,11918.91,2,5,1674.00,115.00
468403,2021-03-03,Inyo,California,1292,37.00,6027,CA,County,18039,7162.26,205.11,205.11,7162.26,3,27,7.00,0.00
59849,2021-03-03,Stanislaus,California,56433,953.00,6099,CA,County,550660,10248.25,173.07,173.07,10248.25,4,10,110.00,3.00
74166,2021-03-03,Tulare,California,48114,768.00,6107,CA,County,466195,10320.57,164.74,164.74,10320.57,5,9,28.00,2.00
27792,2021-03-03,Riverside,California,290498,3849.00,6065,CA,County,2470546,11758.45,155.80,155.80,11758.45,6,6,173.00,20.00
47645,2021-03-03,San Joaquin,California,67123,1150.00,6077,CA,County,762148,8807.08,150.89,150.89,8807.08,7,18,83.00,10.00
424605,2021-03-03,Merced,California,29264,407.00,6047,CA,County,277680,10538.75,146.57,146.57,10538.75,8,8,69.00,2.00
27068,2021-03-03,Fresno,California,95785,1451.00,6019,CA,County,999101,9587.12,145.23,145.23,9587.12,9,14,108.00,0.00
567680,2021-03-03,Kings,California,22091,220.00,6031,CA,County,152940,14444.23,143.85,143.85,14444.23,10,3,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,350.41,14858.59,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,350.41,14858.59,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,350.41,14858.59,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,350.41,14858.59,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,350.41,14858.59,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3502,02/27/21,Lassen,5601,19.00,18320.09,62.15,62.15,18392.05,38,1,0.00,0.00
3503,02/28/21,Lassen,5601,19.00,18320.09,62.15,62.15,18392.05,38,1,0.00,0.00
3504,03/01/21,Lassen,5601,19.00,18320.09,62.15,62.15,18392.05,38,1,0.00,0.00
3505,03/02/21,Lassen,5601,19.00,18320.09,62.15,62.15,18392.05,38,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,350.41,14858.59,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,350.41,14858.59,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,350.41,14858.59,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,350.41,14858.59,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,350.41,14858.59,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3576,02/27/21,Kings,22042,218.00,14412.19,142.54,143.85,14444.23,10,3,39.00,0.00
3577,02/28/21,Kings,22060,220.00,14423.96,143.85,143.85,14444.23,10,3,18.00,2.00
3578,03/01/21,Kings,22082,220.00,14438.34,143.85,143.85,14444.23,10,3,22.00,0.00
3579,03/02/21,Kings,22091,220.00,14444.23,143.85,143.85,14444.23,10,3,9.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)